# Real Data Time Series Forecasting: Model 1 (Bayesian)

## Import Libraries

In [ ]:
import importlib
import utils_model1

importlib.reload(utils_model1)
from utils_model1 import *

In [ ]:
device = cuda_check(verbose=1)

In [ ]:
metadata = pd.read_csv('DeDe_48stations/DeDe_48sites_metadata.csv')

metadata

## Import Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('DeDe_48stations_allfeatures/GHI_CI_NCEP_Iclr_15min_DeDe2023_newiclr.csv', index_col='Datetime', parse_dates=True)

# Drop columns
df = df.drop(columns=['CI_CM', 'CI_RGB', 'Wind direction', 'Snowfall', 'Snow depth'])
df = df.rename(columns={
    'Relative Humidity': 'RelativeHumidity', 
    'Wind speed': 'WindSpeed',
    'Short-wave irradiation': 'Inwp',
    })

df

In [ ]:
df_site1 = df[df['Site_id'] == 1].drop(columns = 'Site_id')

#----- Visualizing Missing Values in DataFrame -----
# Visualize missing data matrix
msno.matrix(df_site1)
plt.title("Missing Data Matrix", fontsize = 20)
plt.show()

''' 
The rightmost line is a sparkline that summarizes the completeness of data in each rows. 
- number '0' means 'there are 0 column that have no missing values in all rows'
- number '10' means 'there are at most 10 columns that have no missing values in the same row'
'''

# Visualize missing data bar chart
msno.bar(df_site1)
plt.title("Missing Data Bar Chart", fontsize = 20)
plt.show()

There are 2 types of missing data (gap):
1. Small gaps: less than 12 hours, we will use 'Linear Interpolation' to fill in the missing values in each column.
2. Large gaps: more than or equal 12 hours, we will leave them as NaN values.

In [ ]:
# Create a full date range
starting_date = min(df_site1.index)
ending_date = max(df_site1.index)
resolution = df_site1.index[1] - df_site1.index[0]

full_date_range = pd.date_range(start=starting_date, end=ending_date, freq=resolution)

# Make index of df_site1 have consistent resolution (where the missing data are filled with NaN to be imputed later)
df_site1 = df_site1.reindex(full_date_range)

# Add Hour index into  the dataframe
df_site1['Hour'] = np.sin(df_site1.index.hour / 24  * np.pi)

df_site1 = df_site1.drop(columns=['CI_CM_interpolated', 'CI_RGB_interpolated', 'rawI'])

# Drop nan rows
df_site1 = df_site1['2023-01-01 07:15:00':]

# Physically, irradiance should not exceed $1200 \text{ W/m}^2$ 
# since it is the solar irradiance that is measured from atmosphere. 
# We can see that there are some values that exceed this limit.
# We will set (or so called 'clip') these values to $1200 \text{ W/m}^2$.

df_site1.loc[:, 'I'] = df_site1['I'].clip(upper=1200)
df_site1.loc[:, 'Inwp'] = df_site1['Inwp'].clip(upper=1200)
df_site1.loc[:, 'Iclr'] = df_site1['Iclr'].clip(upper=1200)

# df_site1['CI_RGB_interpolated'] = df_site1['CI_RGB_interpolated']  / 255
# in the column 'Temperature', the outlier occurs in 2023-01-30 between 04:00 and 10:00.

df_site1.loc['2023-01-30 04:00': '2023-01-30 10:00', 'Temperature'] = np.nan

#  Create a NaN mask
cols = ['I', 'Temperature', 'RelativeHumidity', 'Pressure', 'WindSpeed', 'Rainfall', 'Inwp', 'Iclr']

for col in cols:
    nan_mask = df_site1[col].isna()
    
    # Identify consecutive NaN groups
    group_id = (nan_mask != nan_mask.shift()).cumsum(axis=0)  # Unique ID for each block per column

    # Prepare empty DataFrame to store group sizes per column
    group_sizes = nan_mask.groupby(group_id).transform('sum')

    # Build interpolation mask: interpolate only gaps 
    interpolation_mask = nan_mask & (group_sizes < 48)  # 48 time steps = 48 * 15 minutes = 12 hours

    # Temporarily mask large gaps
    temp_df = df_site1[col].copy()

    # Perform linear time-based interpolation for all columns
    interpolated_df = temp_df.interpolate(method='time', axis=0).where(interpolation_mask)

    # Assign interpolated results back to the original dataframe
    df_site1[col] = df_site1[col].fillna(interpolated_df)

rows_with_nans = df_site1.isna().any(axis=1)
df_site1.loc[rows_with_nans] = np.nan

df_site1

In [ ]:
#----- Visualizing Missing Values in DataFrame -----

import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

# Visualize missing data matrix
msno.matrix(df_site1)
plt.title("Missing Data Matrix", fontsize = 20)
plt.show()

''' 
The rightmost line is a sparkline that summarizes the completeness of data in each rows. 
- number '0' means 'there are 0 column that have no missing values in all rows'
- number '10' means 'there are at most 10 columns that have no missing values in the same row'
'''

# Visualize missing data bar chart
msno.bar(df_site1)
plt.title("Missing Data Bar Chart", fontsize = 20)
plt.show()

In [ ]:
# Visualize the dataset (at site 1)

columns = ['I', 'Inwp', 'Iclr']
columns_name_text = ['I', 'I_nwp', 'I_clr']

# Visualize the result
fig = go.Figure()

# Add the 'Actual' trace

for i in range(len(columns)):
    fig.add_trace(go.Scatter(x=df_site1.index, y=df_site1[columns[i]],
                         mode='lines',
                         name=columns_name_text[i],
                         line=dict(width=1)))

# Update layout for title, axis labels, and grid
fig.update_layout(
    title='Time Series Plot',
    xaxis_title='DateTime',
    yaxis_title='Irradiance',
    hovermode='x unified', # This is useful for time series to see values across traces at a given x
    template='plotly_white' # A clean white background template
)

# Add gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

### EDA: Exploratory Data Analysis

In [ ]:
# Calculate the correlation matrix for all columns
correlation_matrix = df_site1.corr(method='pearson')

# Get the columns to plot against 'I', excluding 'I' itself
columns_to_plot = df_site1.drop(columns=['I']).columns

# Create subplots dynamically
# The number of subplots is equal to the number of columns to plot against 'I'
fig, ax = plt.subplots(1, len(columns_to_plot), figsize=(5 * len(columns_to_plot), 5))

# If there's only one column to plot, ax will not be an array, so handle that case
if len(columns_to_plot) == 1:
    ax = [ax] # Make it a list so the loop works consistently

for i, col in enumerate(columns_to_plot):
    # Calculate the Pearson correlation coefficient between 'I' and the current column
    # We use .loc to ensure we get the specific correlation value
    correlation = correlation_matrix.loc['I', col]

    # Create the scatter plot
    ax[i].scatter(df_site1['I'], df_site1[col], c='black', alpha=0.2)
    
    # Set labels
    ax[i].set_xlabel('$I$')
    ax[i].set_ylabel(f'{col}')
    
    # Set title including the correlation value
    # Using f-string for easy formatting, rounding correlation to 2 decimal places
    ax[i].set_title(f'$I$ vs. {col}\n(Corr: {correlation:.2f})')

plt.tight_layout() # Adjusts subplot parameters for a tight layout
plt.show()

### Make an input and output data

In [ ]:
df_site1

In [ ]:
# Make Input Sequences
lookback = pd.Timedelta('1d')
lookforward = pd.Timedelta('4d')

lookback_timestep = lookback // (df_site1.index[1] - df_site1.index[0])
lookforward_timestep = lookforward // (df_site1.index[1] - df_site1.index[0])
num_feature = df_site1.shape[1]

lag_regressor_cols = ['I']
target_cols=['I']

x_numpy, y_numpy, x_labels, y_labels, datetime = df_transform(df = df_site1, lag_regressor_cols=lag_regressor_cols, target_cols=target_cols, lookback_timedelta=lookback, lookforward_timedelta = lookforward)

x_scaled_numpy, y_scaled_numpy, x_scalers, y_scaler  = scale_transform(x_numpy, y_numpy)

x_scaled = torch.tensor(x_scaled_numpy, dtype=torch.float32).to(device)
y_scaled = torch.tensor(y_scaled_numpy, dtype=torch.float32).to(device)

# Train-Val-Test splitting
train_ratio = 0.8
val_ratio = 0.1

train_samples_count = int(x_scaled.shape[0] * train_ratio)
val_samples_count = int(x_scaled.shape[0] * val_ratio)

x_train_scaled, y_train_scaled, \
x_val_scaled, y_val_scaled, \
x_test_scaled, y_test_scaled, \
datetime_train, datetime_val, datetime_test = split_time_series_data(x = x_scaled, y = y_scaled, train_ratio = train_ratio, val_ratio = val_ratio, datetime=datetime)


# Make a dataloader for train dataset 
train_dataset = TensorDataset(x_train_scaled, y_train_scaled)
val_dataset = TensorDataset(x_val_scaled, y_val_scaled)
test_dataset = TensorDataset(x_test_scaled, y_test_scaled)

train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = False)
val_dataloader = DataLoader(val_dataset, batch_size = 32, shuffle = False)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

# The issue is from shuffling mechanics of torch making data in train_dataloader locates in cpu instead of gpu which we don't want to, so we apply generator manually.
# Create a generator for reproducibility and device consistency
# g = torch.Generator(device=device)
# g.manual_seed(0) # You can set a seed for reproducibility of shuffling
# train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle=True, generator = g)

## Experiment

#### Initial

In [ ]:
torch.manual_seed(42)
model_TransformerEncoder_1 = TransformerEncoderModel(
    d_model=64,
    nhead=4,
    num_layers=5,
    dim_feedforward=256,
    input_size=num_feature,
    output_size=lookforward_timestep,                # make a prediction with the number of timestep equal to lookforward timestep
    seq_len=lookback_timestep,
    dropout=0.1,
)
model_TransformerEncoder_1.to(device)

optimizer = Adam(model_TransformerEncoder_1.parameters(), lr=1e-4)
criterion = nn.L1Loss()

#### Adding an Early stopping and Scheduler into the training process #####
early_stop_count = 0
patience = 20
min_loss = float('inf')

# scheduler = ExponentialLR(optimizer, gamma=0.99)        # To adjust learning rates

print("\n--- Model ---")
print("Model architecture:")
print(model_TransformerEncoder_1)
print(f"Total parameters: {sum(p.numel() for p in model_TransformerEncoder_1.parameters() if p.requires_grad)}")


In [ ]:
##### Train the Model #####

# Clear CUDA cache before starting hyperparameter search
if torch.cuda.is_available():
    torch.cuda.empty_cache()

model_TransformerEncoder_1, *loss_history_model_TransformerEncoder = trainer(
    model_TransformerEncoder_1, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader,
    optimizer=optimizer, 
    criterion=criterion, 
    scheduler=None, 
    max_epochs=200, 
    early_stopping_patience=5,
    )

In [ ]:
##### Make a Prediction to the whole Dataset #####

y_pred, y_test, eval_loss = predictor(
    model=model_TransformerEncoder_1, 
    test_dataloader=test_dataloader,
    y_scaler=y_scaler,
    criterion=nn.L1Loss(), 
    )

print(f'Evaluation Loss: {eval_loss}')

plotly_visualization_line(x=datetime_test, y=[y_test[:, 0], y_pred[:, 0]], name=['Actual', 'Predicted'])

In [ ]:
##### Make a Prediction to the whole Dataset #####

y_train_pred, y_train, eval_loss = predictor(
    model=model_TransformerEncoder_1, 
    test_dataloader=train_dataloader,
    y_scaler=y_scaler,
    criterion=nn.L1Loss(), 
    )

print(f'Evaluation Loss: {eval_loss}')

plotly_visualization_line(x=datetime_train, y=[y_train[:, 0], y_train_pred[:, 0]], name=['Actual', 'Predicted'])

#### Bayesian Optimization (Encoder) with Optuna

In [ ]:
def objective_bayesian(trial, model_parameters, train_dataloader, val_dataloader, device=cuda_check()):
    if torch.cuda.is_available():
        gc.collect()
        torch.cuda.empty_cache()

    try:
        d_model_per_head = trial.suggest_categorical('d_model_per_head', [4, 8, 16, 32, 64])
        nhead = trial.suggest_categorical('nhead', [1, 2, 4, 8])
        num_layers = trial.suggest_int('num_layers', 1, 4)
        dim_feedforward = trial.suggest_categorical('dim_feedforward', [128, 256, 512, 1024])
            
        model = TransformerEncoderModel(
            input_size=model_parameters['num_feature'],
            output_size=model_parameters['lookforward_timestep'],
            seq_len=model_parameters['lookback_timestep'],
            d_model=d_model_per_head * nhead,
            nhead=nhead,
            num_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=0.1
        ).to(device)
        optimizer = Adam(model.parameters(), lr=0.001)
        criterion = nn.L1Loss()
        # scheduler = ExponentialLR(optimizer, gamma=0.99)

        _, _, val_loss = trainer(
            model=model,
            optimizer=optimizer,
            criterion=criterion,
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            scheduler=None,
            max_epochs=50,
            early_stopping_patience=20,
            display_result=False
        )

        return val_loss[-1] if val_loss else float('inf')

    except Exception as e:
        print(f"Trial failed: {e}")
        return float('inf')

In [ ]:
# Bayesian Optimization
n_trials = 60

model_parameters = {
    'num_feature': num_feature, 
    'lookforward_timestep': lookforward_timestep, 
    'lookback_timestep': lookback_timestep
}

objective = lambda trial: objective_bayesian(
        trial, 
        model_parameters, 
        train_dataloader, 
        val_dataloader
    )
 

study = run_optimization(
    objective = objective, 
    n_trials = n_trials, 
    timeout = None, 
    sampler = optuna.samplers.TPESampler(seed=42), 
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,     # First 5 trials will run fully without begin pruned
        n_warmup_steps=3,       # No pruning in the first 3 steps of any trial
        interval_steps=1,       # Pruning check every 2 epochs
        n_min_trials=5          # Need at least 3 completed trials before pruning starts
        ),
    n_jobs = 1,
    )

final_model_bayesian, r2_bayesian, mse_bayesian, mae_bayesian = train_final_model(study, model_parameters, train_dataloader=train_dataloader, val_dataloader=val_dataloader, test_dataloader=test_dataloader)
best_n_models_baesian = show_n_best_models(study, n=5)
print("\nStudy Summary:")
print(f"Total Trials: {len(study.trials)}")
print(f"Pruned Trials: {sum(t.state == optuna.trial.TrialState.PRUNED for t in study.trials)}")
print(f"Completed Trials: {sum(t.state == optuna.trial.TrialState.COMPLETE for t in study.trials)}")

In [ ]:
num_best_model = 5
best_n_models_baesian = show_n_best_models(study, n=num_best_model)

In [ ]:
best_n_models_baesian[0].params

In [ ]:
model_parameters

In [ ]:
##### Make a Prediction to the whole Dataset #####

y_pred, y_test, eval_loss = predictor(
    model=final_model_bayesian, 
    test_dataloader=test_dataloader,
    y_scaler=y_scaler,
    criterion=nn.L1Loss(), 
    )

print(f'Evaluation Loss: {eval_loss}')

plotly_visualization_line(x=datetime_test, y=[y_test[:, 0], y_pred[:, 0]], name=['Actual', 'Predicted'])

In [ ]:
##### Make a Prediction to the whole Dataset #####

y_pred, y_test, eval_loss = predictor(
    model=final_model_bayesian, 
    test_dataloader=train_dataloader,
    y_scaler=y_scaler,
    criterion=nn.L1Loss(), 
    )

print(f'Evaluation Loss: {eval_loss}')

plotly_visualization_line(x=datetime_test, y=[y_test[:, 0], y_pred[:, 0]], name=['Actual', 'Predicted'])